In [64]:
%reset -f

import numpy as np
import numba as  nb
import pandas as pd

all = pd.read_parquet('tmp.parq')
# df_plt = bid_ask.copy()[bid_ask.t > np.datetime64('2023-06-22 10:00:00')]
# m = 15
# fig = Figure(layout=go.Layout(height=500, margin={'t':m,'b':m,'l':m,'r':m}))
# fig.add_traces(Scatter(x=df_plt['t'], y=df_plt['ask'], name='ask', mode="markers", marker={'size':2},showlegend=False))
# fig.add_traces(Scatter(x=df_plt['t'], y=df_plt['bid'], name='bid', mode="markers", marker={'size':2},showlegend=False))
# fig.add_traces(Scatter(x=df_plt['t'], y=(df_plt['bid'] + df_plt['ask'])/2, name='mid', mode="markers", marker={'size':2, 'color':'black'},showlegend=False))
# fig.show()
subset = all[-10000:]

In [65]:
@nb.jit(nopython=True)
def insert(a:np.array, idx:np.int64, v:np.int64) -> np.array:
    N = len(a)
    M = N + 1

    if idx < 0:
        if np.abs(idx) > M:
            raise Exception(f"Insertion index out of range {idx}.")
    else:
        if np.abs(idx) > N:
            raise Exception(f"Insertion index out of range {idx}.")

    # standard python behaviour. allows insertions from rear    
    insertion_pt = idx % M

    tgt = np.full(M, v)
        
    for i in range (0, insertion_pt):
        tgt[i] = a[i]
    tgt[insertion_pt] = v
    for i in range (insertion_pt, N):
        tgt[i+1] = a[i]

    return tgt


In [93]:

@nb.jit(nopython=True)
def do_a_manual(t:np.array, val:np.array, bin_start:np.array, bin_end:np.array, ) -> np.array:
    n_ticks, n_bins = len(t), len(np.unique(bin_start))
    i, j, bi = 0, 0, 0
    bin_start_index = np.full(n_bins, t[0])
    bin_end_index = np.full(n_bins, t[0])
    n_samples = np.full(n_bins, 0)
    o = np.full(n_bins, 0.0)
    h = np.full(n_bins, 0.0)
    l = np.full(n_bins, 0.0)
    c = np.full(n_bins, 0.0)
    twap = np.full(n_bins, 0.0)
    t0 = np.full(n_bins, t[0])
    t1 = np.full(n_bins, t[0])

    while j < n_ticks:
        while (j < n_ticks) & (bin_start[i] == bin_start[j]):
            j += 1
        # i is the index of the first element on the binned timestamp
        # je is the index to the last element of the binned timestamp
        # ii is the insertion point for the current bin

        bin_start_index[bi] = bin_start[j - 1]
        bin_end_index[bi]   = bin_end[j - 1]

        n_samples[bi] = j - i

        times_bin = t[i:j]
        ticks_bin = val[i:j]

        o[bi] = ticks_bin[0]
        h[bi] = np.max(ticks_bin)
        l[bi] = np.min(ticks_bin)
        c[bi] = ticks_bin[-1]
        t0[bi] = times_bin[0]
        t1[bi] = times_bin[-1]

        # #####
        # for some calcs, forward fill from prev bin and to end of bin
        # #####
        if bi > 0:
            #  if there is a prev bin
            times_bin = insert(times_bin, 0, bin_start_index[bi-1])
            ticks_bin = insert(ticks_bin, 0, val[i-1])

        times_bin = insert(times_bin, -1, bin_start_index[bi-1])

        dt = np.diff(ticks_bin)
        # twap[bi] = np.sum(dt * val)/np.sum(dt)

        bi += 1
        i = j
            
    return bin_start_index, bin_end_index, t0, t1, n_samples, o, h, l, c

#pad_on_bin_bopundary
def foo(ticks:pd.DataFrame,
        bin_size_sec:int,
        time_col:str = "t",
        ) -> pd.DataFrame:

    bin_start = ticks[time_col].values.astype(f'datetime64[{bin_size_sec}s]').astype('datetime64[ns]')
    bin_end = (bin_start + np.timedelta64(bin_size_sec, 's')).astype(np.int64)
    bin_start  = bin_start.astype(np.int64)
    
    tt = ticks.t.values.astype(np.int64)
    mid = (ticks.bid.values + ticks.ask.values) / 2
    bin_start_index, bin_end_index, t0, t1, n_samples, o, h, l, c =  \
        do_a_manual(tt, mid, bin_start, bin_end)
    return pd.DataFrame({'bin_start':bin_start_index.astype('datetime64[ns]'),
                         'bin_end':bin_end_index.astype('datetime64[ns]'),
                         't0':t0.astype('datetime64[ns]'),
                         't1':t1.astype('datetime64[ns]'),
                         'n_samples':n_samples,
                         'o':o, 'h':h, 'l':l, 'c':c})

res = foo(subset, 60)
res[:5]

,bin_start,bin_end,t0,t1,n_samples,o,h,l,c
0,2023-06-29 13:37:00,2023-06-29 13:38:00,2023-06-29 13:37:51.031,2023-06-29 13:37:59.866,8,1.159215,1.159235,1.159215,1.159235
1,2023-06-29 13:38:00,2023-06-29 13:39:00,2023-06-29 13:38:01.006,2023-06-29 13:38:59.517,51,1.159255,1.159275,1.159105,1.159185
2,2023-06-29 13:39:00,2023-06-29 13:40:00,2023-06-29 13:39:00.632,2023-06-29 13:39:56.961,44,1.159195,1.159255,1.159135,1.159255
3,2023-06-29 13:40:00,2023-06-29 13:41:00,2023-06-29 13:40:00.261,2023-06-29 13:40:59.552,42,1.159265,1.159305,1.159245,1.159275
4,2023-06-29 13:41:00,2023-06-29 13:42:00,2023-06-29 13:41:00.652,2023-06-29 13:41:59.447,39,1.159265,1.159355,1.159175,1.159185


In [68]:
res = foo(all, 60)
res

,bin_start,t0,t1,n_samples,o,h,l,c
0,2023-06-06 19:22:00,2023-06-06 19:22:24.585,2023-06-06 19:22:59.563,18,1.162185,1.162205,1.162185,1.162195
1,2023-06-06 19:23:00,2023-06-06 19:23:01.584,2023-06-06 19:23:20.241,7,1.162195,1.162205,1.162165,1.162165
2,2023-06-06 19:27:00,2023-06-06 19:27:35.485,2023-06-06 19:27:53.589,4,1.162165,1.162175,1.162165,1.162165
3,2023-06-06 19:29:00,2023-06-06 19:29:01.237,2023-06-06 19:29:05.443,3,1.162165,1.162185,1.162165,1.162185
4,2023-06-06 20:27:00,2023-06-06 20:27:22.878,2023-06-06 20:27:37.094,6,1.161995,1.162005,1.161995,1.161995
...,...,...,...,...,...,...,...,...
14394,2023-06-29 17:58:00,2023-06-29 17:58:00.342,2023-06-29 17:58:55.936,36,1.160345,1.160375,1.160245,1.160325
14395,2023-06-29 17:59:00,2023-06-29 17:59:00.737,2023-06-29 17:59:58.962,34,1.160305,1.160325,1.160265,1.160305
14396,2023-06-29 18:00:00,2023-06-29 18:00:00.432,2023-06-29 18:00:59.638,41,1.160285,1.160355,1.160275,1.160275
14397,2023-06-29 18:01:00,2023-06-29 18:01:00.077,2023-06-29 18:01:58.659,33,1.160285,1.160345,1.160275,1.160295
